# Assignment 2: Entity Resolution (Part 1)

## Objective

Data scientists often spend 80% of their time on [data preparation](https://www.infoworld.com/article/3228245/the-80-20-data-science-dilemma.html). If your career goal is to become a data scientist, you have to master data cleaning and data integration skills. In this assignment, you will learn how to solve the Entity Resolution (ER) problem, a very common problem in data cleaning and integration. After completing this assignment, you should be able to answer the following questions:

1. What is ER?
2. What are the applications of ER in data integration and cleaning? 
3. How to avoid $n^2$ comparisons? 
4. How to compute Jaccard Similarity?
5. How to evaluate an ER result?

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

The data for Assignment 2 (Part 1 and Part 2) are in `A2-data.zip`.

## Overview

ER is defined as finding different records that refer to the same real-world entity, e.g., iPhone 4-th generation vs. iPhone four. It is central to data integration and cleaning. In this assignment, you will learn how to apply ER in a data integration setting. But the program that you are going to write can be easily extended to a data-cleaning setting, being used to detect _duplication values_.   

Imagine that you want to help your company's customers to buy products at a cheaper price. In order to do so, you first write a [web scraper](https://nbviewer.jupyter.org/github/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1-1.ipynb) to crawl product data from Amazon.com and Google Shopping, respectively, and then integrate the data together. Since the same product may have different representations in the two websites, you are facing an ER problem. 

Existing ER techniques can be broadly divided into two categories: similarity-based (Part 1) and learning-based (Part 2). 


## Similarity Join

Unlike a learning-based technique, a similarity-based technique (a.k.a similarity join) does not need any label data. It first chooses a similarity function and a threshold, and then returns the record pairs whose similarity values are above the threshold. These returned record pairs are thought of as matching pairs, i.e., referring to the same real-world entity. 

Depending on particular applications, you may need to choose different similarity functions. In this assignment, we will use Jaccard similarity, i.e., $\textsf{Jaccard}(r, s) = \big|\frac{r \cap s}{r \cup s}\big|$. Here is the formal definition of this problem.

> **Jaccard-Similarity Join**: Given two DataFrames, R and S, and a threshold $\theta \in (0, 1]$, the jaccard-similarity join problem aims to find all record pairs $(r, s) \in R \times S$ such that $\textsf{Jaccard}(r, s) \geq \theta$  

To implement similarity join, you need to address the following challenges:

1. Jaccard is used to quantify the similarity between two sets instead of two records. You need to convert each record to a set.

2. A naive implementation of similarity join is to compute Jaccard for all $|R \times S|$ possible pairs. Imagine R and S have one million records. This requires doing 10^12 pair comparisons, which is extremely expensive. Thus, you need to know how to avoid n^2 comparisons. 

3. The output of ER is a set of matching pairs, where each pair is considered as referring to the same real-world entity. You need to know how to evaluate the quality of an ER result.

Next, you will be guided to complete four tasks. After finishing these tasks, I suggest you going over the above challenges again, and understand how they are addressed.

Read the code in `similarity_join.py` first, and then implement the remaining four functions: `preprocess_df`, `filtering`, `verification`, and `evaluate` by doing Tasks A-D, respectively.



The program will output the following when running on the sample data:


> Before filtering: 256 pairs in total

> After Filtering: 84 pairs left

> After Verification: 6 similar pairs

> (precision, recall, fmeasure) =  (1.0, 0.375, 0.5454545454545454)


### Task A. Data Preprocessing (Record --> Token Set)

Since Jaccard needs to take two sets as input, your first job is to preprocess DataFrames by transforming each record into a set of tokens. Please implement the following function.   

```python
def preprocess_df(self, df, cols): 
    """
    Input:
        df: a pandas DataFrame
        cols: a list of column names in df to be concatenated and tokenized

    Output:
        new_df: a new DataFrame that is the same as df, but with an extra column:
                - joinKey: a list of tokens for each record
    """
    
    # Make a copy so we do not change the original dataframe
    new_df = df.copy()

    def build_joinkey(row):
        # Join the selected columns into one string
        concat_str = " ".join(
            str(row[c]) if pd.notna(row[c]) else ""
            for c in cols
        )

        # Split the string into tokens and convert to lower-case
        tokens = [t.lower() for t in re.split(r'\W+', concat_str) if t != ""]

        return tokens

    # Apply the function to every row to create joinKey
    new_df["joinKey"] = new_df.apply(build_joinkey, axis=1)

    return new_df
```

In [1]:
from similarity_join import SimilarityJoin

er = SimilarityJoin("A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_sample.csv", "A2-data/part1-similarity-join/Amazon-Google-Sample/Google_sample.csv")

amazon_cols = ["title", "manufacturer"]
google_cols = ["name", "manufacturer"]

df1 = er.preprocess_df(er.df1, amazon_cols)
df2 = er.preprocess_df(er.df2, google_cols)

# joinKey가 잘 만들어졌는지 확인
df1[["id", "joinKey"]].head()

,id,joinKey
0,b0002mg5uk,"[iview, mediapro, 2, 5, global, marketing]"
1,b0002jtvng,"[bias, deck, le, 3, 5, macintosh, cd, bias]"
2,b0007lw22g,"[apple, ilife, 06, mac, dvd, older, version, a..."
3,b00007kh02,"[extensis, intellihance, pro, 4, x, win, mac, ..."
4,b000saufpw,"[dk, amazing, animals, 1, 1, global, software,..."


For the purpose of testing, you can compare your outputs with new_df1 and new_df2 that can be found from the `Amazon-Google-Sample` folder.

### Task B. Filtering Obviously Non-matching Pairs

To avoid $n^2$ pair comparisons, ER algorithms often follow a filtering-and-verification framework. The basic idea is to first filter obviously non-matching pairs and then only verify the remaining pairs.  

In Task B, your job is to implement the <font color="blue">filtering</font> function. This function will filter all the record pairs whose joinKeys do not share any token. This is because based on the definition of Jaccard, we can deduce that **if two sets do not share any element (i.e., $r\cap s = \phi$), their Jaccard similarity values must be zero**. Thus, we can safely remove them. 

```python
def filtering(self, df1, df2):
    """
    Input: df1, df2 have a 'joinKey' column (list of tokens) and an 'id' column
    Output: cand_df with columns: id1, joinKey1, id2, joinKey2
            Keep only pairs that share at least one token
    """

    # Build an inverted index for df1
    # token -> list of row indices in df1
    inv_index = {}

    for i, tokens in enumerate(df1["joinKey"]):
        # use set() to avoid duplicate tokens in one record
        for t in set(tokens):
            if t not in inv_index:
                inv_index[t] = []
            inv_index[t].append(i)

    # Find candidate pairs using df2
    # use set to remove duplicate (i, j) pairs
    cand_pairs = set()

    for j, tokens2 in enumerate(df2["joinKey"]):
        for t in set(tokens2):
            if t in inv_index:
                for i in inv_index[t]:
                    cand_pairs.add((i, j))

    # Build candidate DataFrame
    rows = []
    
    for i, j in cand_pairs:
        rows.append({
            "id1": df1.iloc[i]["id"],
            "joinKey1": df1.iloc[i]["joinKey"],
            "id2": df2.iloc[j]["id"],
            "joinKey2": df2.iloc[j]["joinKey"]
        })

    cand_df = pd.DataFrame(rows, columns=["id1", "joinKey1", "id2", "joinKey2"])

    return cand_df
```

In [1]:
from similarity_join import SimilarityJoin
import pandas as pd

er = SimilarityJoin(
    "A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_sample.csv",
    "A2-data/part1-similarity-join/Amazon-Google-Sample/Google_sample.csv"
)

amazon_cols = ["title", "manufacturer"]
google_cols = ["name", "manufacturer"]

df1 = er.preprocess_df(er.df1, amazon_cols)
df2 = er.preprocess_df(er.df2, google_cols)

cand_df = er.filtering(df1, df2)

cand_df.head()

,id1,joinKey1,id2,joinKey2
0,b000saufpw,"[dk, amazing, animals, 1, 1, global, software,...",http://www.google.com/base/feeds/snippets/1102...,"[punch, software, 20100, punch, 5, in, 1, home..."
1,b000saufpw,"[dk, amazing, animals, 1, 1, global, software,...",http://www.google.com/base/feeds/snippets/1761...,"[onone, software, essentials, for, adobe, phot..."
2,b000v7vz1u,"[onone, essentials, for, adobe, photoshop, ele...",http://www.google.com/base/feeds/snippets/7249...,"[onone, software, ice, 10770, intellihance, pr..."
3,b000v7vz1u,"[onone, essentials, for, adobe, photoshop, ele...",http://www.google.com/base/feeds/snippets/1020...,"[palmspring, software, 523, oxford, american, ..."
4,b000jx1kma,"[aircraft, power, pack, for, ms, flight, simul...",http://www.google.com/base/feeds/snippets/1761...,"[onone, software, essentials, for, adobe, phot..."


In [8]:
gt_cand = pd.read_csv(
    "A2-data/part1-similarity-join/Amazon-Google-Sample/cand_df.csv"
)

gt_cand.head()

,id1,joinKey1,id2,joinKey2
0,b0002mg5uk,"['iview', 'mediapro', '2', '5', 'global', 'mar...",http://www.google.com/base/feeds/snippets/1267...,"['iview', 'mediapro', '2', '6', 'media', 'mana..."
1,b0002mg5uk,"['iview', 'mediapro', '2', '5', 'global', 'mar...",http://www.google.com/base/feeds/snippets/1761...,"['onone', 'software', 'essentials', 'for', 'ad..."
2,b000v7vz1u,"['onone', 'essentials', 'for', 'adobe', 'photo...",http://www.google.com/base/feeds/snippets/1267...,"['iview', 'mediapro', '2', '6', 'media', 'mana..."
3,b000v7vz1u,"['onone', 'essentials', 'for', 'adobe', 'photo...",http://www.google.com/base/feeds/snippets/1761...,"['onone', 'software', 'essentials', 'for', 'ad..."
4,b0002mg5uk,"['iview', 'mediapro', '2', '5', 'global', 'mar...",http://www.google.com/base/feeds/snippets/1102...,"['punch', 'software', '20100', 'punch', '5', '..."


In [9]:
print("My cand_df:", len(cand_df))
print("GT cand_df:", len(gt_cand))

My cand_df: 84
GT cand_df: 84


For the purpose of testing, you can compare your output with cand_df that can be found from the `Amazon-Google-Sample` folder.

### Task C. Computing Jaccard Similarity for Survived Pairs

In the second phase of the filtering-and-verification framework, we will compute the Jaccard similarity for each survived pair and return those pairs whose jaccard similarity values are no smaller than the specified threshold.

In Task C, your job is to implement the <font color="blue">verification</font> function. This task looks simple, but there are a few small "traps". 


```python
def verification(self, cand_df, threshold):
    """
    Input:
        cand_df: the output DataFrame from filtering()
                 It has columns: id1, joinKey1, id2, joinKey2
        threshold: float in (0, 1]

    Output:
        result_df: a new DataFrame with columns:
                   id1, joinKey1, id2, joinKey2, jaccard
                   Keep only rows with jaccard >= threshold
    """

    rows = []

    for _, row in cand_df.iterrows():
        tokens1 = row["joinKey1"]
        tokens2 = row["joinKey2"]

        # Jaccard is defined on sets (remove duplicates)
        set1 = set(tokens1)
        set2 = set(tokens2)

        inter = set1.intersection(set2)
        union = set1.union(set2)

        # avoid division by zero (just in case)
        jacc = 0.0 if len(union) == 0 else len(inter) / len(union)

        # keep only pairs whose jaccard is no smaller than threshold
        if jacc >= threshold:
            rows.append({
                "id1": row["id1"],
                "joinKey1": row["joinKey1"],
                "id2": row["id2"],
                "joinKey2": row["joinKey2"],
                "jaccard": jacc
            })

    result_df = pd.DataFrame(rows, columns=["id1", "joinKey1", "id2", "joinKey2", "jaccard"])
    return result_df
```

In [3]:
cand_df = er.filtering(df1, df2)
result_df = er.verification(cand_df, 0.5)

print("My result_df:", len(result_df))

gt_result = pd.read_csv("A2-data/part1-similarity-join/Amazon-Google-Sample/result_df.csv")
print("GT result_df:", len(gt_result))


My result_df: 6
GT result_df: 6


For the purpose of testing, you can compare your output with result_df that can be found from the `Amazon-Google-Sample` folder.

### Task D. Evaluating an ER result

How should we evaluate an ER result? Before answering this question, let's first recall what the ER result looks like. The goal of ER is to identify all matching record pairs. Thus, the ER result should be a set of identified matching pairs, denoted by R. One thing that we want to know is that what percentage of the pairs in $R$ that are truly matching? This is what Precision can tell us. Let $T$ denote the truly matching pairs in $R$. Precision is defined as:
$$Precision = \frac{|T|}{|R|}$$

In addition to Precision, another thing that we care about is that what percentage of truly matching pairs that are identified. This is what Recall can tell us. Let $A$ denote the truly matching pairs in the entire dataset. Recall is defined as: 

$$Recall = \frac{|T|}{|A|}$$

There is an interesting trade-off between Precision and Recall. As more and more pairs that are identified as matching, Recall increases while Precision potentially decreases. For the extreme case, if we return all the pairs as matching pairs, we will get a perfect Recall (i.e., Recall = 100%) but precision will be the worst. Thus, to balance Precision and Recall, people often use FMeasure to evaluate an ER result:

$$FMeasure = \frac{2*Precision*Recall}{Precision+Recall}$$

In Task D, you will be given an ER result as well as the ground truth that tells you what pairs are truly matching. Your job is to calculate Precision, Recall and FMeasure for the result. 
```python
def evaluate(self, result, ground_truth):
    """
    Input:
        result: a list of matching pairs found by your ER algorithm
                e.g., [["a1","g1"], ["a2","g2"], ...]
        ground_truth: a list of true matching pairs (human-labeled)

    Output:
        (precision, recall, fmeasure)
    """

    # Make them sets of tuples so we can compare easily
    # (Also removes duplicates automatically)
    result_set = set(tuple(p) for p in result)
    gt_set = set(tuple(p) for p in ground_truth)

    # True positives = pairs that appear in BOTH result and ground truth
    true_positive = result_set.intersection(gt_set)

    # Compute precision and recall (handle divide-by-zero)
    precision = len(true_positive) / len(result_set) if len(result_set) > 0 else 0.0
    recall = len(true_positive) / len(gt_set) if len(gt_set) > 0 else 0.0

    # Compute F-measure (harmonic mean)
    fmeasure = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

    return (precision, recall, fmeasure)
```


In [4]:
import pandas as pd

# ✅ 정답 매칭표 로드 (파일명이 다르면 그 파일명으로)
gt = pd.read_csv("A2-data/part1-similarity-join/Amazon-Google-Sample/Amazon_Google_perfectMapping_sample.csv")

# (정답표 컬럼명이 idAmazon / idGoogle이면)
gt_pairs = gt.rename(columns={"idAmazon":"id1", "idGoogle":"id2"})[["id1","id2"]]

# ✅ 내 결과 (result_df가 이미 있다고 가정)
my_pairs = result_df[["id1","id2"]].copy()

# ✅ 내가 맞춘 것(교집합)
hit = my_pairs.merge(gt_pairs, on=["id1","id2"], how="inner")

print("내가 찾은 매칭 수:", len(my_pairs))
print("정답 매칭 수:", len(gt_pairs))
print("맞춘 매칭 수(True Positive):", len(hit))

hit.head(20)


내가 찾은 매칭 수: 6
정답 매칭 수: 16
맞춘 매칭 수(True Positive): 6


,id1,id2
0,b000licg1m,http://www.google.com/base/feeds/snippets/6070...
1,b000v7vz1u,http://www.google.com/base/feeds/snippets/1761...
2,b000ndibge,http://www.google.com/base/feeds/snippets/1227...
3,b000jx1kma,http://www.google.com/base/feeds/snippets/1835...
4,b00002s6sc,http://www.google.com/base/feeds/snippets/1102...
5,b000ndibbo,http://www.google.com/base/feeds/snippets/1693...


In [5]:
missed = gt_pairs.merge(my_pairs, on=["id1","id2"], how="left", indicator=True)
missed = missed[missed["_merge"]=="left_only"].drop(columns=["_merge"])

print("놓친 정답 수:", len(missed))
missed.head(20)


놓친 정답 수: 10


,id1,id2
0,b0002mg5uk,http://www.google.com/base/feeds/snippets/1267...
1,b0002jtvng,http://www.google.com/base/feeds/snippets/5572...
2,b00007kh02,http://www.google.com/base/feeds/snippets/7249...
3,b00006hvvo,http://www.google.com/base/feeds/snippets/3409...
4,b0007lw22g,http://www.google.com/base/feeds/snippets/1084...
5,b000saufpw,http://www.google.com/base/feeds/snippets/1007...
10,b000in8n30,http://www.google.com/base/feeds/snippets/4377...
11,b0000vyk1o,http://www.google.com/base/feeds/snippets/1740...
13,b000gaqlxe,http://www.google.com/base/feeds/snippets/1479...
14,b00013wh0w,http://www.google.com/base/feeds/snippets/1020...


## Submission

Implement `preprocess_df`, `filtering`, `verification`, and `evaluate` functions in `similarity_join.py`. Submit your code file (`similarity_join.py`) to Canvas.